In [2]:
import matplotlib.pyplot as plt
import numpy as np
import gymnasium as gym
import sys
sys.path.append('..')
from PIL import Image
from tqdm import trange
import numpy as np
import matplotlib.pyplot as plt
import os
from plot_utils import *
from utils import *
from agent_env_utils import *
from config import color_maps, background_quadrants
import warnings
from scene_helpers import get_random_position, create_config, create_directories, finalize_quadrant



In [8]:


# Define directories
data_dir = "../benchmark/quantity/discrete_counting"
final_dir = create_directories(data_dir, empty=True)

# Initialize pair of colors
color_keys = list(color_maps.keys())
random.seed(42)
random.shuffle(color_keys)

# Define ranges and object parameters
object_shapes = ["sphere", "cube"]
object_size_ranges = {
    "sphere": (0.02, 0.05),
    "cube": (0.02, 0.05),
}
num_objects_range = (1, 7)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")

    outputs = []
    for background in trange(4):
        for color_idx in range(5):  # Adjust the range as needed
            color = color_keys[color_idx]
            for r in range(2):  # Increase if you want more examples per color and background
                
                # Generate random configurations for objects
                num_objects = np.random.randint(*num_objects_range)
                configs = []
                object_types = []
                for _ in range(num_objects):
                    obj_type = random.choice(object_shapes)
                    object_types.append(obj_type)
                    position = get_random_position(background_quadrants, background, scale=1)
                    configs.append(create_config(obj_type, color_maps[color], object_size_ranges[obj_type], position))

                configs_collate = collate_infos(configs)

                env.register_configures(configs_collate)
                
                _ = env.reset(options={"reconfigure": True})

                env.initialize_objects(background=background)
                obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))

                save_dir_final = f"{final_dir}/{color}_bg{background}_num{num_objects}_r{r}.png"
                collect_and_save(env, save_dir_final, steps=1, mode=[f"front{background}", f"side{background}", f"top{background}"])
                poses_final = env.get_important_obj_poses()
                
                outputs.append({
                    "source": '/'.join(save_dir_final.split('/')[1:-1]),
                    "color": color,
                    "num_objects": num_objects,
                    "background": background,
                    "round": r,
                    "sizes": [np.round(size, 4).tolist() for size in configs_collate['size']],
                    #"poses_final": [np.round(item, 4).tolist() for item in poses_final],
                })

    write_out(f"{data_dir}/index.jsonl", outputs)

del env

[2024-08-31 04:03:26.644] [svulkan2] [error] GLFW error: X11: Failed to open display :5
[2024-08-31 04:03:26.644] [svulkan2] [warning] Continue without GLFW.
100%|██████████| 4/4 [01:01<00:00, 15.27s/it]
